# YouTube 及多平台下載方案 - 實際測試

測試三種下載方案在不同平台的實際表現：

1. **pytubefix** - YouTube 專用
2. **yt-dlp** - 多平台通用
3. **Playwright** - 瀏覽器模擬（備選）

## 測試 URL 列表

- YouTube MP3: https://www.youtube.com/watch?v=0rp3pP2Xwhs
- YouTube MP4: https://www.youtube.com/shorts/vGmdigoUJ6c
- Facebook Video: https://www.facebook.com/share/r/1Bx14m8c7b/
- Instagram Video: https://www.instagram.com/reel/DRtrroMkeDU/
- X (Twitter) Video: https://x.com/Appraiser008/status/1995090648689717458

In [ ]:
import sys
import os
from pathlib import Path
import time
from datetime import datetime

# 設置輸出目錄
OUTPUT_DIR = Path('./downloads')
OUTPUT_DIR.mkdir(exist_ok=True)

# 定義測試 URL
TEST_URLS = {
    'youtube_mp3': 'https://www.youtube.com/watch?v=0rp3pP2Xwhs',
    'youtube_mp4': 'https://www.youtube.com/shorts/vGmdigoUJ6c',
    'facebook': 'https://www.facebook.com/share/r/1Bx14m8c7b/',
    'instagram': 'https://www.instagram.com/reel/DRtrroMkeDU/?utm_source=ig_web_copy_link&igsh=NTc4MTIwNjQ2YQ==',
    'twitter': 'https://x.com/Appraiser008/status/1995090648689717458?s=20',
}

print(f"輸出目錄: {OUTPUT_DIR.absolute()}")
print(f"\n測試 URL:")
for platform, url in TEST_URLS.items():
    print(f"  {platform:15}: {url}")

# 測試結果記錄
test_results = {}

## 方案 1: pytubefix 測試

pytubefix 是 pytube 的維護版本，測試 YouTube 下載性能

In [ ]:
# 安裝 pytubefix
!pip install -q pytubefix

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import subprocess
import re

def test_pytubefix_youtube_mp3():
    """測試 pytubefix YouTube MP3 下載"""
    print("\n" + "="*80)
    print("🧪 測試 1: pytubefix - YouTube MP3 下載")
    print("="*80)
    
    url = TEST_URLS['youtube_mp3']
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 建立 YouTube 物件
        yt = YouTube(url, on_progress_callback=on_progress)
        
        print(f"標題: {yt.title}")
        print(f"長度: {yt.length} 秒")
        print(f"作者: {yt.author}")
        
        # 清理檔名
        safe_title = re.sub(r'[<>:"/\\|?*]', '_', yt.title)
        
        # 下載音訊
        audio_stream = yt.streams.get_audio_only()
        print(f"音訊串流: {audio_stream}")
        
        output_file = audio_stream.download(
            output_path=str(OUTPUT_DIR),
            filename=f"{safe_title}.m4a"
        )
        
        print(f"✅ 下載完成 (m4a): {output_file}")
        
        # 轉換為 MP3
        try:
            mp3_file = str(OUTPUT_DIR / f"{safe_title}.mp3")
            subprocess.run(
                ['ffmpeg', '-i', output_file, '-q:a', '0', '-map', 'a', mp3_file],
                capture_output=True,
                check=True
            )
            Path(output_file).unlink()
            print(f"✅ 轉換成功 (MP3): {mp3_file}")
            
            elapsed_time = time.time() - start_time
            file_size = Path(mp3_file).stat().st_size / (1024 * 1024)
            
            test_results['pytubefix_youtube_mp3'] = {
                'status': '✅ 成功',
                'time': f"{elapsed_time:.1f}s",
                'size': f"{file_size:.2f}MB",
                'file': mp3_file
            }
            
            return True
            
        except (FileNotFoundError, subprocess.CalledProcessError) as e:
            print(f"⚠️ ffmpeg 不可用: {e}")
            test_results['pytubefix_youtube_mp3'] = {
                'status': '⚠️ 部分成功 (m4a)',
                'file': output_file
            }
            return False
            
    except Exception as e:
        print(f"❌ 下載失敗: {e}")
        test_results['pytubefix_youtube_mp3'] = {
            'status': f'❌ 失敗: {str(e)[:50]}'
        }
        import traceback
        traceback.print_exc()
        return False

def test_pytubefix_youtube_mp4():
    """測試 pytubefix YouTube MP4 下載（最高品質）"""
    print("\n" + "="*80)
    print("🧪 測試: pytubefix - YouTube MP4 下載")
    print("="*80)
    
    url = TEST_URLS['youtube_mp4']
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 建立 YouTube 物件
        yt = YouTube(url, on_progress_callback=on_progress)
        
        print(f"標題: {yt.title}")
        print(f"長度: {yt.length} 秒")
        
        # 清理檔名
        safe_title = re.sub(r'[<>:"/\\|?*]', '_', yt.title)
        
        # 取得最高品質的視訊（包含音訊）
        # 先找最高品質的 progressive（自帶音訊）流
        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
        
        if stream is None:
            # 如果沒有 progressive，則分別下載視訊和音訊後合併
            print("⚠️ 沒有 progressive 流，嘗試分別下載視訊和音訊...")
            
            # 下載最高品質視訊
            video_stream = yt.streams.filter(mime_type='video/mp4').order_by('resolution').desc().first()
            audio_stream = yt.streams.filter(mime_type='audio/mp4').order_by('abr').desc().first()
            
            if video_stream is None or audio_stream is None:
                print("❌ 找不到適合的視訊或音訊流")
                test_results['pytubefix_youtube_mp4'] = {'status': '❌ 失敗: 找不到視訊或音訊流'}
                return False
            
            print(f"視訊: {video_stream}")
            print(f"音訊: {audio_stream}")
            
            # 下載視訊
            video_file = video_stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}_video.mp4"
            )
            
            # 下載音訊
            audio_file = audio_stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}_audio.m4a"
            )
            
            # 合併視訊和音訊
            final_file = str(OUTPUT_DIR / f"{safe_title}.mp4")
            cmd = [
                'ffmpeg',
                '-i', video_file,
                '-i', audio_file,
                '-c:v', 'copy',
                '-c:a', 'aac',
                '-strict', 'experimental',
                final_file
            ]
            
            subprocess.run(cmd, capture_output=True, check=True)
            
            # 刪除臨時文件
            Path(video_file).unlink()
            Path(audio_file).unlink()
            
            print(f"✅ 合併成功: {final_file}")
            
        else:
            print(f"✅ 找到最高品質流: {stream}")
            
            final_file = stream.download(
                output_path=str(OUTPUT_DIR),
                filename=f"{safe_title}.mp4"
            )
            
            print(f"✅ 下載完成: {final_file}")
        
        elapsed_time = time.time() - start_time
        file_size = Path(final_file).stat().st_size / (1024 * 1024)
        
        test_results['pytubefix_youtube_mp4'] = {
            'status': '✅ 成功',
            'time': f"{elapsed_time:.1f}s",
            'size': f"{file_size:.2f}MB",
            'file': final_file
        }
        
        return True
        
    except Exception as e:
        print(f"❌ 下載失敗: {e}")
        test_results['pytubefix_youtube_mp4'] = {
            'status': f'❌ 失敗: {str(e)[:50]}'
        }
        import traceback
        traceback.print_exc()
        return False

# 執行測試
test_pytubefix_youtube_mp3()

## 方案 2: yt-dlp 多平台測試

測試 yt-dlp 在各平台的下載能力

In [ ]:
from yt_dlp import YoutubeDL
from pathlib import Path

def convert_mp4_for_mobile(input_file, output_file=None, target_bitrate='1000k', target_height=720):
    """
    轉碼 MP4 以適配手機分享
    
    參數:
        input_file: 輸入 MP4 文件
        output_file: 輸出文件 (default: input_file + _mobile.mp4)
        target_bitrate: 目標比特率 (default: 1000k)
        target_height: 目標高度 (default: 720p)
    """
    input_path = Path(input_file)
    if output_file is None:
        output_file = str(input_path.parent / f"{input_path.stem}_mobile.mp4")
    
    print(f"\n📹 轉碼 MP4 以適配手機分享...")
    print(f"   輸入: {input_path.name}")
    print(f"   輸出: {Path(output_file).name}")
    print(f"   目標: {target_height}p @ {target_bitrate}")
    
    try:
        cmd = [
            'ffmpeg',
            '-i', str(input_file),
            '-vf', f'scale=-2:{target_height}',  # 保持縱橫比，高度為 target_height，寬度必須能被 2 整除
            '-b:v', target_bitrate,
            '-b:a', '128k',
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-crf', '23',  # 質量 (0-51, 預設 23, 越低越好)
            '-preset', 'medium',  # 編碼速度
            output_file
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            output_size = Path(output_file).stat().st_size / (1024 * 1024)
            print(f"✅ 轉碼成功: {output_size:.2f}MB")
            return output_file
        else:
            print(f"❌ 轉碼失敗: {result.stderr[:200]}")
            return None
            
    except subprocess.TimeoutExpired:
        print("❌ 轉碼超時")
        return None
    except Exception as e:
        print(f"❌ 轉碼錯誤: {e}")
        return None

def test_ytdlp_platform(platform_name, url, enable_mobile_conversion=False):
    """測試 yt-dlp 非 YouTube 平台"""
    print(f"\n🧪 測試: yt-dlp - {platform_name}")
    print("-" * 80)
    
    start_time = time.time()
    
    try:
        print(f"URL: {url}\n")
        
        # 優先選擇有視訊的 MP4 格式
        ydl_opts = {
            'format': 'best[ext=mp4][vcodec!^=none][acodec!^=none]/best[vcodec!^=none][acodec!^=none]/best',
            'outtmpl': str(OUTPUT_DIR / '%(title)s.%(ext)s'),  # 添加副檔名
            'http_headers': {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            },
            'quiet': False,
            'no_warnings': False,
        }
        
        if 'instagram' in platform_name.lower():
            ydl_opts['extractor_args'] = {
                'instagram': {'fetch_all_comments': False}
            }
        
        with YoutubeDL(ydl_opts) as ydl:
            print("開始下載...")
            info = ydl.extract_info(url, download=True)
            
            # 獲取下載的文件
            ext = info.get('ext', 'mp4')
            filename = f"{info.get('title', 'video')}.{ext}"
            filepath = OUTPUT_DIR / filename
            
            elapsed_time = time.time() - start_time
            
            print(f"\n✅ 下載成功:")
            print(f"   標題: {info.get('title', 'N/A')}")
            print(f"   時間: {elapsed_time:.1f}s")
            print(f"   文件: {filename}")
            print(f"   副檔名: .{ext}")
            
            # MP4 轉碼以適配手機分享
            mobile_file = None
            if enable_mobile_conversion and ext == 'mp4':
                mobile_file = convert_mp4_for_mobile(str(filepath))
            
            test_results[f'ytdlp_{platform_name.replace(" ", "_").lower()}'] = {
                'status': '✅ 成功',
                'title': info.get('title', 'N/A'),
                'time': f"{elapsed_time:.1f}s",
                'file': filename,
                'mobile_file': Path(mobile_file).name if mobile_file else None
            }
            
            return True
            
    except Exception as e:
        elapsed_time = time.time() - start_time
        error_msg = str(e)[:100]
        print(f"❌ 下載失敗: {error_msg}")
        
        test_results[f'ytdlp_{platform_name.replace(" ", "_").lower()}'] = {
            'status': f'❌ 失敗: {error_msg}'
        }
        
        return False

# yt-dlp 只用於非 YouTube 平台
print("\n" + "="*80)
print("🧪 方案 2: yt-dlp (Facebook、Instagram、X/Twitter)")
print("="*80)

## 多平台測試

按順序測試各個平台

In [ ]:
# 首先用 pytubefix 下載 YouTube MP4
print("\n【方案 1: pytubefix - YouTube】")
test_pytubefix_youtube_mp4()

# 然後用 yt-dlp 下載其他平台（啟用手機轉碼）
print("\n【方案 2: yt-dlp - 其他平台】")
print("\n" + "-"*80)
test_ytdlp_platform('Facebook', TEST_URLS['facebook'], enable_mobile_conversion=True)

print("\n" + "-"*80)
test_ytdlp_platform('Instagram', TEST_URLS['instagram'], enable_mobile_conversion=True)

print("\n" + "-"*80)
test_ytdlp_platform('X (Twitter)', TEST_URLS['twitter'], enable_mobile_conversion=True)